# Preprocessing ERA5 data for SUMMA forcings in the MeuseA

TODO:
 * Get air pressure
 * Get longwave radiation

In [1]:
%pylab inline

import os
import salem
import xarray as xr

from datetime import datetime, timedelta
from calendar import monthrange
import matplotlib.pyplot as plt
import metsim.disaggregate as disagg
import metsim.constants as cnst

era5_raw_dir = '/mnt/data/ERA-5/raw/'
shapefile = '../summa_era5_setup/shapefiles/meuse_hydrosheds.shp'

Populating the interactive namespace from numpy and matplotlib


## Step 1: Load data
We start by reading the shapefile, the raw data, and then cropping out a bounding box to reduce the data size

In [2]:
# cut out ROI from ERA5 data using shapefile
shdf = salem.read_shapefile(shapefile)
x_bounds = float(np.round(shdf.min_x - 3)), float(np.round(shdf.max_x + 3))
y_bounds = float(np.round(shdf.max_y + 3)), float(np.round(shdf.min_y - 3))

# open ERA5 forcing file
ds = xr.open_mfdataset(era5_raw_dir + '*.nc')
# transform longitude to West negative, East positive
ds = ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180))
ds = ds.sel(latitude=slice(*y_bounds), longitude=slice(*x_bounds))

## Step 2: Compute masks and put data on HRU dimension
Because SUMMA requires the input to be aranged spatially by the HRU dimension, we have to crop out all of the non-active cells in the input. From this we restack on our new dimension

In [3]:
# Coordinate reference system for both STATSGO and ERA5:
mask = ds['t2m'].isel(time=0).salem.roi(shape=shdf, all_touched=True) > 0
mask = mask.stack(hru=['latitude', 'longitude'])

era_ds = ds.stack(hru=['latitude', 'longitude'])
era_ds = era_ds.where(mask, drop=True)
mask = mask.where(mask, drop=True)

lons = mask.longitude
lats = mask.latitude
mask['hru'].values = np.arange(len(mask['hru'])) + 1
era_ds['hru'].values = np.arange(len(mask['hru'])) + 1

summa_ds = xr.Dataset(coords={'hru': mask['hru'].values, 'time': ds['time'].values})
summa_ds['latitude'] = lats
summa_ds['longitude'] = lons
summa_ds

/mnt/home/user7/.conda/envs/fair_hydro/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in greater
  return func(*args2)


<xarray.Dataset>
Dimensions:    (hru: 101, time: 8784)
Coordinates:
  * hru        (hru) int64 1 2 3 4 5 6 7 8 9 10 ... 93 94 95 96 97 98 99 100 101
  * time       (time) datetime64[ns] 2000-01-01 ... 2000-12-31T23:00:00
Data variables:
    latitude   (hru) float64 51.75 51.75 51.75 51.75 ... 48.25 48.0 48.0 48.0
    longitude  (hru) float64 4.75 5.0 5.25 5.5 5.75 ... 5.75 6.0 5.5 5.75 6.0

## Add in the variables and write out

In [4]:
# Wind speed
summa_ds['windspd'] = (np.sqrt(era_ds['u10']**2 + era_ds['v10']**2) 
                       * 4.87/np.log(67.8*10-5.42)) # check this logarithmic profile!

In [6]:
# Air temperature
summa_ds['airtemp'] = era_ds['t2m']

In [7]:
# Shortwave radiation
summa_ds['SWRadAtm'] = era_ds['ssrd'] / 3600

In [8]:
# CHANGE TO AIR PRESSURE - sp
summa_ds['airpres'] = era_ds['msl']

In [10]:
# Specific humidity
E = 611*np.exp((17.76*(era_ds['d2m']-cnst.KELVIN))/(era_ds['d2m']-29.65)) # actual vapour pressure (Pa)
spechum = disagg.specific_humidity(E/1000,summa_ds['airpres']/1000) #[SUMMA: g/g, MetSim output is g/kg?]
summa_ds['spechum'] = spechum

In [13]:
# Cloud cover
era_ds['tcc'] = era_ds['t2m']
era_ds['tcc'].values = np.random.random(size=era_ds['t2m'].shape)

/mnt/home/user7/.conda/envs/fair_hydro/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in greater
  return func(*args2)


In [14]:
# Longwave
params = {'lw_type': 'SATTERLUND', 'lw_cloud': 'CLOUD_DEARDORFF'}
longwave = disagg.longwave(era_ds['t2m']-cnst.KELVIN,E/1000,era_ds['tcc'], params)  # [SUMMA: W/m2]
summa_ds['LWRadAtm'] = longwave

In [16]:
# Precipitation
summa_ds['pptrate'] = era_ds['tp']
summa_ds['pptrate'] = summa_ds['pptrate'].where(era_ds['tp']>0.000001).fillna(0) * 1000 / 3600

In [19]:
summa_ds.to_netcdf('../summa_era5_setup/forcing/meuse_testdata.nc')